In [2]:
import pickle
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
import glob as glob
import os
import sys; sys.path.append("../../..")  # Allows access to all the scripts/modules in the larger directory
from utils import calc_loadings
from collections import defaultdict
from sklearn.cross_decomposition import CCA


### Load consolidated dimreduc dataframe

In [3]:
#path = '/home/marcush/Data/TsaoLabData/neural_control_output/decoding_deg_final_analysis_230809_140453_Alfie/decoding_deg_final_analysis_230809_140453_Alfie_glom.pickle'
path = '/home/marcush/Data/TsaoLabData/neural_control_output/testing_230322_214006_Jamie_dimreduc/testing_230322_214006_Jamie_dimreduc_glom.pickle'

with open(path, 'rb') as f:
    dat = pickle.load(f) 
df_dimreduc = pd.DataFrame(dat)

### Load the processed data from both regions

In [4]:
def make_hashable(obj):
    if isinstance(obj, dict):
        return tuple(sorted((key, make_hashable(value)) for key, value in obj.items()))
    elif isinstance(obj, (list, tuple)):
        return tuple(make_hashable(item) for item in obj)
    else:
        return obj


unique_hashes = set(make_hashable(d) for d in df_dimreduc['full_arg_tuple'])
unique_dicts = [dict(u) for u in unique_hashes]

In [5]:
preload_dict_path = df_dimreduc['data_path'][0] + "/preloaded/preloadDict.pickle"

with open(preload_dict_path, 'rb') as file:
    preloadDict = pickle.load(file)


for arg_dict in unique_dicts:
    arg_tuple = tuple(sorted(arg_dict.items()))


    for args in preloadDict.keys():

        if args == arg_tuple:

            preloadID = preloadDict[arg_tuple]
            loaded_data_path = os.path.dirname(preload_dict_path) + f"/preloaded_data_{preloadID}.pickle"
            
            if arg_dict['region'] == 'AM':
                with open(loaded_data_path, 'rb') as file:
                    print(args)
                    AM_loaded_data = pickle.load(file)

            elif arg_dict['region'] == 'ML':
                with open(loaded_data_path, 'rb') as file:
                    print(args)
                    ML_loaded_data = pickle.load(file)


(('bin_width', 50), ('boxcox', 0.5), ('data_path', '/home/marcush/Data/TsaoLabData/split/degraded/degraded_only_v4_AnalysisDegraded_230322_214006_Jamie.pickle'), ('degraded', True), ('filter_fn', 'none'), ('filter_kwargs', ()), ('manual_unit_selection', (361, 379, 338, 460, 138, 542, 196, 307, 410, 512, 418, 89, 359, 491, 443, 493, 441, 230, 429, 530, 422, 310, 523, 296, 442, 417, 223, 124, 486, 546, 90, 269, 82, 120, 363, 344, 75, 519, 462, 378, 458, 91, 234, 174, 123, 199, 560, 376, 313, 286, 302, 245, 425, 244, 84, 350, 76, 327, 149, 12, 277, 211, 189, 276, 264, 477, 448, 509, 534, 203, 267, 470, 154, 209, 541, 266, 438, 449, 249, 380, 274, 92, 108, 13, 298, 31, 358, 569, 96, 253, 484, 391, 197, 252, 155, 168, 557, 437, 432, 454, 372, 433, 255, 446, 407, 236, 232, 495, 105, 399, 131, 175, 384, 440, 395, 128, 7, 32, 504, 10, 205, 23, 254, 554, 178, 387, 15, 371, 78, 565, 38, 27, 258, 343, 65, 444, 370, 294, 250, 134, 389, 471, 113, 25, 242, 285, 551, 24, 488, 172, 430, 164, 548, 180,

### Sum spike rate per trial

In [6]:
AM_spikes = np.sum(AM_loaded_data['spike_rates'], 1)
ML_spikes = np.sum(ML_loaded_data['spike_rates'], 1)

### Fit CCA

In [9]:
manual_CCA_dim = 39 #np.inf 

cca_dims = np.min((manual_CCA_dim, AM_spikes.shape[-1], ML_spikes.shape[-1])).astype(int)

ccamodel = CCA(n_components=cca_dims)
ccamodel.fit(ML_spikes, AM_spikes)

cca_save_path = os.path.dirname(df_dimreduc['results_file'][0]) + f"/CCA_{cca_dims}_dims.pickle"

with open(cca_save_path, 'wb') as file:
    pickle.dump(ccamodel, file)

ValueError: Found input variables with inconsistent numbers of samples: [1991, 2336]